## Install Staff tables 
     Steps to install Staff tables and do Staff Assignment tables 

## Import dependencies

In [1]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
from datetime import date
pymysql.install_as_MySQLdb()

## Import configuration variables

In [2]:
# import sys
# sys.path.append('../../../../')
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

## Set up `simple_salesforce`

Pass in the username, password, and security token here. The security token contains information about your org.

In [3]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

## Connect to your MySQL database

In [4]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Query Data

In [5]:
staff_data = pd.read_sql("SELECT * FROM staff", conn)
staff_data.head(30)

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


## Transform Data

In [6]:
staff_data = staff_data.rename(columns={'ID_Staff':'ID_Staff__c',
                                          'EmployeeID':'EmployeeID__c',
                                          'LastName':'Lastname__c',
                                          'FirstName':'Firstname__c',
                                          'MiddleName':'Middlename__c',
                                          'BirthDate':'Birthdate__c'
                                         })

staff_data

,ID_Staff__c,EmployeeID__c,Lastname__c,Firstname__c,Middlename__c,Birthdate__c
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


In [7]:
staff_data['Birthdate__c'] = pd.to_datetime(staff_data['Birthdate__c']).dt.date

staff_data.head()

,ID_Staff__c,EmployeeID__c,Lastname__c,Firstname__c,Middlename__c,Birthdate__c
0,1,000184220,Wimberly,Sam,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Popelka,Sarah,Nisan,1994-06-07


## Convert DataFrame to a list of dictionaries 

This can help facilitate a bulk insert

In [8]:
staff_records = staff_data.to_dict('records')
staff_records

[{'ID_Staff__c': 1,
  'EmployeeID__c': '000184220',
  'Lastname__c': 'Wimberly',
  'Firstname__c': 'Sam',
  'Middlename__c': 'Nico',
  'Birthdate__c': datetime.date(1995, 7, 5)},
 {'ID_Staff__c': 2,
  'EmployeeID__c': '000130109',
  'Lastname__c': 'Sanford',
  'Firstname__c': 'Gemini',
  'Middlename__c': 'Blair',
  'Birthdate__c': datetime.date(1992, 4, 22)},
 {'ID_Staff__c': 3,
  'EmployeeID__c': '000160655',
  'Lastname__c': 'Williams',
  'Firstname__c': 'Dartanion',
  'Middlename__c': 'De Angelo',
  'Birthdate__c': datetime.date(1993, 5, 21)},
 {'ID_Staff__c': 4,
  'EmployeeID__c': '000159108',
  'Lastname__c': 'Popelka',
  'Firstname__c': 'Sarah',
  'Middlename__c': 'Nisan',
  'Birthdate__c': datetime.date(1994, 6, 7)}]

## Insert records to staff table

In [ ]:
for rec in staff_records:

    record = {
        'ID_Staff__c': rec['ID_Staff__c'],
        'EmployeeID__c': rec['EmployeeID__c'],
        'Lastname__c': rec['Lastname__c'],
        'Firstname__c': rec['Firstname__c'],
        'Middlename__c':rec['Middlename__c'],
        'Birthdate__c': str(rec['Birthdate__c'])
    }
    
    try:
        sf.staff__c.create(record)
    except Exception as e:
        print(e)

## Create Staff Lookup Table
Use this later to crosswalk the course code with the primary key from the `staff` table

The query **Salesforce** to retrieve the record IDs 

In [9]:
staff_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT EmployeeID__c, Name FROM staff__c")
for row in data:
    rec = {
        'staffrecord_ID__c': row['Name'], # this is a critical line of code
        'EmployeeID__c': row['EmployeeID__c']
    }
    staff_lookup_list.append(rec)
    
staff_lookup_list

[{'staffrecord_ID__c': 'a073h000002uHds', 'EmployeeID__c': '000160655'},
 {'staffrecord_ID__c': 'a073h000002uHdi', 'EmployeeID__c': '000184220'},
 {'staffrecord_ID__c': 'a073h000002uHdx', 'EmployeeID__c': '000159108'},
 {'staffrecord_ID__c': 'a073h000002uHdn', 'EmployeeID__c': '000130109'}]

In [10]:
staff_lookup_df = pd.DataFrame(staff_lookup_list)
staff_lookup_df

,staffrecord_ID__c,EmployeeID__c
0,a073h000002uHds,000160655
1,a073h000002uHdi,000184220
2,a073h000002uHdx,000159108
3,a073h000002uHdn,000130109


## Create class Lookup Table

In [11]:
class_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
data = sf.query_all_iter("SELECT Section__c, Name FROM class__c")
for row in data:
    rec = {
        'Class Record ID ': row['Name'],
        'Section__c': row['Section__c']
    }
    class_lookup_list.append(rec)
class_lookup_list

[{'Class Record ID ': 'a083h000000nn4o', 'Section__c': 'GWARL201905UIUX3'},
 {'Class Record ID ': 'a083h000000nn4Z', 'Section__c': 'GWU-ARL-DATA-PT-09-0'},
 {'Class Record ID ': 'a083h000000nn4j', 'Section__c': 'GWARL201905WEB3'},
 {'Class Record ID ': 'a083h000000nn4e', 'Section__c': 'GWDC201805DATA3'}]

In [13]:
class_lookup_df = pd.DataFrame(class_lookup_list)
class_lookup_df = class_lookup_df.rename(columns={'Class Record ID':'Class_Record_ID__c'})
class_lookup_df

,Class Record ID,Section__c
0,a083h000000nn4o,GWARL201905UIUX3
1,a083h000000nn4Z,GWU-ARL-DATA-PT-09-0
2,a083h000000nn4j,GWARL201905WEB3
3,a083h000000nn4e,GWDC201805DATA3


In [14]:
class_lookup_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Class Record ID   4 non-null      object
 1   Section__c        4 non-null      object
dtypes: object(2)
memory usage: 192.0+ bytes


## SQL Query to pull staff assignment data

In [15]:
query = '''
    SELECT
        sa.*
        ,c.Section
        ,s.EmployeeID
    FROM
        staffassignment sa
        INNER JOIN class c
        ON c.ID_class  = sa.ID_class
        INNER Join staff s 
        on s.ID_staff = sa.ID_staff
'''
staffassignment_data_df = pd.read_sql(query, conn)
staffassignment_data_df

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate,Section,EmployeeID
0,1,1,1,Teacher Assistant,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000184220
1,2,4,1,Teacher Assistant,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000159108
2,3,3,1,Instructor,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000160655


In [17]:
staffassignment_data_df = staffassignment_data_df.rename(columns={
                                                                'ID_StaffAssignment':'ID_StaffAssignment__c',
                                                                'ID_Staff':'ID_Staff__c',
                                                                'ID_Class':'ID_Class__c',
                                                                'Role':'Role__c',
                                                                'StartDate':'StartDate__c',
                                                                'EndDate':'EndDate__c',
                                                                'Section':'Section__c',
                                                                'EmployeeID':'EmployeeID__c',   
})
staffassignment_data_df

,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Role__c,StartDate__c,EndDate__c,Section__c,EmployeeID__c
0,1,1,1,Teacher Assistant,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000184220
1,2,4,1,Teacher Assistant,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000159108
2,3,3,1,Instructor,2020-03-16,None,GWU-ARL-DATA-PT-09-0,000160655


In [18]:
staffassignment_data_df['StartDate__c'] = pd.to_datetime(staffassignment_data_df['StartDate__c']).dt.date
staffassignment_data_df['EndDate__c'] = pd.to_datetime(staffassignment_data_df['EndDate__c']).dt.date
staffassignment_data_df

,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Role__c,StartDate__c,EndDate__c,Section__c,EmployeeID__c
0,1,1,1,Teacher Assistant,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,000184220
1,2,4,1,Teacher Assistant,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,000159108
2,3,3,1,Instructor,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,000160655


## Merges - Pull in Staff Record and Class Record to prep for upload

In [19]:
sa_merge_df = pd.merge(staffassignment_data_df,staff_lookup_df , how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','Course__c','Section__c','StartDate__c','Enddate__c','coursecode__c'], inplace=True)

sa_merge_df

,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Role__c,StartDate__c,EndDate__c,Section__c,EmployeeID__c,staffrecord_ID__c
0,1,1,1,Teacher Assistant,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,000184220,a073h000002uHdi
1,2,4,1,Teacher Assistant,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,000159108,a073h000002uHdx
2,3,3,1,Instructor,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,000160655,a073h000002uHds


In [20]:
sa_merge_df = pd.merge(sa_merge_df,class_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','Course__c','Section__c','StartDate__c','Enddate__c','coursecode__c'], inplace=True)

sa_merge_df

,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Role__c,StartDate__c,EndDate__c,Section__c,EmployeeID__c,staffrecord_ID__c,Class Record ID
0,1,1,1,Teacher Assistant,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,000184220,a073h000002uHdi,a083h000000nn4Z
1,2,4,1,Teacher Assistant,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,000159108,a073h000002uHdx,a083h000000nn4Z
2,3,3,1,Instructor,2020-03-16,NaT,GWU-ARL-DATA-PT-09-0,000160655,a073h000002uHds,a083h000000nn4Z


## Drop unnecessary columns

In [21]:
sa_final_df= sa_merge_df.drop(columns = ['EmployeeID__c','EndDate__c','Section__c'])

sa_final_df


,ID_StaffAssignment__c,ID_Staff__c,ID_Class__c,Role__c,StartDate__c,staffrecord_ID__c,Class Record ID
0,1,1,1,Teacher Assistant,2020-03-16,a073h000002uHdi,a083h000000nn4Z
1,2,4,1,Teacher Assistant,2020-03-16,a073h000002uHdx,a083h000000nn4Z
2,3,3,1,Instructor,2020-03-16,a073h000002uHds,a083h000000nn4Z


## Prep data to load - Make list of dictionaries, then load to Salesforce

In [22]:
sa_records = sa_final_df.to_dict('records')
sa_records

[{'ID_StaffAssignment__c': 1,
  'ID_Staff__c': 1,
  'ID_Class__c': 1,
  'Role__c': 'Teacher Assistant',
  'StartDate__c': datetime.date(2020, 3, 16),
  'staffrecord_ID__c': 'a073h000002uHdi',
  'Class Record ID ': 'a083h000000nn4Z'},
 {'ID_StaffAssignment__c': 2,
  'ID_Staff__c': 4,
  'ID_Class__c': 1,
  'Role__c': 'Teacher Assistant',
  'StartDate__c': datetime.date(2020, 3, 16),
  'staffrecord_ID__c': 'a073h000002uHdx',
  'Class Record ID ': 'a083h000000nn4Z'},
 {'ID_StaffAssignment__c': 3,
  'ID_Staff__c': 3,
  'ID_Class__c': 1,
  'Role__c': 'Instructor',
  'StartDate__c': datetime.date(2020, 3, 16),
  'staffrecord_ID__c': 'a073h000002uHds',
  'Class Record ID ': 'a083h000000nn4Z'}]

## Insert Data to Salesforce

Loop through the DataFrame and assemble indiviudal dictionaries

In [26]:
for rec in sa_records:
 
    record = {
        'ID_StaffAssignment__c': rec['ID_StaffAssignment__c'],
        'ID_Staff__c': rec['ID_Staff__c'],
        'Role__c': rec['Role__c'],
        'StartDate__c': str(rec['StartDate__c']),
        'staffrecord_ID__c': rec['staffrecord_ID__c'],
        'Class Record ID ': rec['Class_Record_ID__c' ]
    }
    
    try:
        sf.staffassignment__c.create(record)
    except Exception as e:
        print(e)

KeyError: 'Class_Record_ID__c'

## Example of inserting rows in bulk

In [ ]:
try:
    sf.bulk.Lead.insert(student_load)
except Exception as e:
    print(e)